Chapter 35
Debugging Pandas

In [ ]:
# deletes variables
%reset -f

In [ ]:
import pandas as pd
#url = 'https://raw.githubusercontent.com/mattharrison/datasets/master/data/dirtydevil.txt'
url = 'https://github.com/mattharrison/datasets/raw/master/data/dirtydevil.txt'

In [ ]:
df = pd.read_csv(url,skiprows=lambda num: num < 34 or num == 35,
    sep='\t')

#df2= pd.read_csv('dirtydevil.txt',skiprows=lambda num: num < 34 or num == 35, sep='\t')

In [ ]:
df

In [ ]:
df.head(10)

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
def to_denver_time(df_, time_col, tz_col):
    return (df_
       .assign(**{tz_col: df_[tz_col].replace('MDT', 'MST7MDT')})
       .groupby(tz_col)
       [time_col]
       .transform(lambda s: pd.to_datetime(s)
           .dt.tz_localize(s.name, ambiguous=True)
           .dt.tz_convert('America/Denver'))
    )
def tweak_river(df_):
    return (df_
     .assign(datetime=to_denver_time(df_, 'datetime', 'tz_cd'))
     .rename(columns={'144166_00060': 'cfs',
                      '144167_00065': 'gage_height'})
    )


    

In [ ]:
dd = tweak_river(df)

In [ ]:
dd2 = pd.read_json(dd.to_json())
dd.equals(dd2)

In [ ]:
# quantify the count of different values

( dd
 .ne(dd2)
 .sum()
)

In [ ]:
# percent differences
(dd
 .ne(dd2)
 .mean()
 .mul(100)
)

In [ ]:
pd.testing.assert_frame_equal(dd, dd2)

In [ ]:
pd.testing.assert_frame_equal(dd,
                              (dd2
                                  .assign(datetime=dd2.datetime
                                          .dt.tz_localize('UTC')
                                           .dt.tz_convert('America/Denver'))
                              )
                             )

In [ ]:
(dd.equals(dd2
            .assign(datetime=dd2.datetime
                                          .dt.tz_localize('UTC')
                                           .dt.tz_convert('America/Denver'))
           )
)

In [ ]:
pd.testing.assert_frame_equal(dd,
                              (dd2
                                  .assign(datetime=dd2.datetime
                                          .dt.tz_localize('UTC')
                                           .dt.tz_convert('America/Denver'))
                              ),
                              check_exact=True
                             )

In [ ]:
dd[dd.cfs.ne(dd2.cfs)]

In [ ]:
dd.iloc[96246].cfs,dd2.iloc[96246].cfs 

In [ ]:
(dd.round(2).equals(
                    dd2
                        .assign(datetime=dd2.datetime.dt.tz_localize('UTC').dt.tz_convert('America/Denver'))
                        .round(2)                              
                   )
)

In [ ]:
# Function to help diagnose where dataframes are not the same
# https://github.com/mattharrison/effective_pandas_book/blob/main/35-debugging-code.ipynb


def cmp_dfs(df1, df2, round_amt=3):
    diff_cols = set(df1.columns) ^ set(df2.columns)
    if diff_cols:
        print(f'Different columns {diff_cols}')
    if df1.shape != df2.shape:
        print(f'Different shapes {df1.shape} {df2.shape}')
    bad = False
    for col in df1.columns:
        s1 = df1[col]
        s2 = df2[col]
        if s1.equals(s2):
            continue
        bad = True            
        if s1.dtype != s2.dtype:
            print(f'{col} types differ {s1.dtype} vs {s2.dtype}')
        if s1.dtype == float:
            if s1.round(round_amt).equals(s2.round(round_amt)):
                print(f'{col} has rounding differences'
                      f'{df1[s1.ne(s2)][col].dropna().iloc[0]} '
                      f'vs {df2[s1.ne(s2)][col].dropna().iloc[0]}')
        else:
            print(f'{col} differs {df1[s1.ne(s2)][col].dropna()}')
    if not bad:
        print('Same')

In [ ]:
cmp_dfs(dd, dd2)

35.2 Debugging Chains



In [ ]:
import pandas as pd
autos = pd.read_csv('https://github.com/mattharrison/datasets/raw/'
    'master/data/vehicles.csv.zip')
def to_tz(df_, time_col, tz_offset, tz_name):
    return (df_
             .groupby(tz_offset)
             [time_col]
             .transform(lambda s: pd.to_datetime(s)
                 .dt.tz_localize(s.name, ambiguous=True)
                 .dt.tz_convert(tz_name))
            )

In [ ]:
def tweak_autos(autos):
    cols = ['city08', 'comb08', 'highway08', 'cylinders',
            'displ', 'drive', 'eng_dscr', 'fuelCost08',
            'make', 'model', 'trany', 'range', 'createdOn',
            'year']
    return (autos
     [cols]
     .assign(cylinders=autos.cylinders.fillna(0).astype('int8'),
             displ=autos.displ.fillna(0).astype('float16'),
             drive=autos.drive.fillna('Other').astype('category'),
             automatic=autos.trany.str.contains('Auto'),
             speeds=autos.trany.str.extract(r'(\d)+').fillna('20')
                    .astype('int8'),
             offset=autos.createdOn
                    .str.extract(r'\d\d:\d\d ([A-Z]{3}?)')
                    .replace('EDT', 'EST5EDT'),
             str_date=(autos.createdOn.str.slice(4,19) + ' ' +
                       autos.createdOn.str.slice(-4)),
             createdOn=lambda df_: to_tz(df_, 'str_date',
                       'offset', 'America/New_York'),
             ffs=autos.eng_dscr.str.contains('FFS')
            )
     .astype({'highway08': 'int8', 'city08': 'int16',
              'comb08': 'int16', 'fuelCost08': 'int16',
              'range': 'int16',  'year': 'int16',
              'make': 'category'})
     .drop(columns=['trany', 'eng_dscr'])
    )

In [ ]:
tweak_autos(autos)

35.3 Debugging chains Part 2

In [ ]:
from IPython.display import display, HTML
def show(df_, rows=20, cols=30, title=None):
    if title:
        display(HTML(f'<h2>{title}</h2>'))
    with pd.option_context('display.min_rows', rows,
                           'display.max_columns', cols):
        display(df_)
    return df_

In [ ]:
def get_var(df, var_name):
    globals()[var_name] = df
    return df

add the method show to tweek

In [ ]:
def tweak_autos(autos):
    cols = ['city08', 'comb08', 'highway08', 'cylinders',
            'displ', 'drive', 'eng_dscr', 'fuelCost08',
            'make', 'model', 'trany', 'range', 'createdOn',
            'year']
    return (autos
     [cols]
     .assign(cylinders=autos.cylinders.fillna(0).astype('int8'),
             displ=autos.displ.fillna(0).astype('float16'),
             drive=autos.drive.fillna('Other').astype('category'),
             automatic=autos.trany.str.contains('Auto'),
             speeds=autos.trany.str.extract(r'(\d)+').fillna('20')
                    .astype('int8'),
             offset=autos.createdOn
                    .str.extract(r'\d\d:\d\d ([A-Z]{3}?)')
                    .replace('EDT', 'EST5EDT'),
             str_date=(autos.createdOn.str.slice(4,19) + ' ' +
                       autos.createdOn.str.slice(-4)),
             createdOn=lambda df_: to_tz(df_, 'str_date',
                       'offset', 'America/New_York'),
             ffs=autos.eng_dscr.str.contains('FFS')
            )
     .pipe(show, rows=2 , title='New Columns')
     .astype({'highway08': 'int8', 'city08': 'int16',
              'comb08': 'int16', 'fuelCost08': 'int16',
              'range': 'int16',  'year': 'int16',
              'make': 'category'})
     .drop(columns=['trany', 'eng_dscr'])
    )

In [ ]:
# call tweek_autos() with show

tweak_autos(autos)

35.4 Debugging Chains Part III

In [ ]:
def tweak_autos(autos):
    cols = ['city08', 'comb08', 'highway08', 'cylinders',
            'displ', 'drive', 'eng_dscr', 'fuelCost08',
            'make', 'model', 'trany', 'range', 'createdOn',
            'year']
    return (autos
     [cols]
     .assign(cylinders=autos.cylinders.fillna(0).astype('int8'),
             displ=autos.displ.fillna(0).astype('float16'),
             drive=autos.drive.fillna('Other').astype('category'),
             automatic=autos.trany.str.contains('Auto'),
             speeds=autos.trany.str.extract(r'(\d)+').fillna('20')
                    .astype('int8'),
             offset=autos.createdOn
                    .str.extract(r'\d\d:\d\d ([A-Z]{3}?)')
                    .replace('EDT', 'EST5EDT'),
             str_date=(autos.createdOn.str.slice(4,19) + ' ' +
                       autos.createdOn.str.slice(-4)),
             createdOn=lambda df_: to_tz(df_, 'str_date',
                       'offset', 'America/New_York'),
             ffs=autos.eng_dscr.str.contains('FFS')
            )
     .pipe(get_var, 'new_cols')
     .astype({'highway08': 'int8', 'city08': 'int16',
              'comb08': 'int16', 'fuelCost08': 'int16',
              'range': 'int16',  'year': 'int16',
              'make': 'category'})
     .drop(columns=['trany', 'eng_dscr'])
    )

In [ ]:
# call tweek_autos() with get_var()

res = tweak_autos(autos)


35.5 Debugging Chains Part IV

In [ ]:
def err(*args):
    1/0

In [ ]:
def tweak_autos(autos):
    cols = ['city08', 'comb08', 'highway08', 'cylinders',
            'displ', 'drive', 'eng_dscr', 'fuelCost08',
            'make', 'model', 'trany', 'range', 'createdOn',
            'year']
    return (autos
     [cols]
     .assign(cylinders=autos.cylinders.fillna(0).astype('int8'),
             displ=autos.displ.fillna(0).astype('float16'),
             drive=autos.drive.fillna('Other').astype('category'),
             automatic=autos.trany.str.contains('Auto'),
             speeds=autos.trany.str.extract(r'(\d)+').fillna('20')
                    .astype('int8'),
             offset=autos.createdOn
                    .str.extract(r'\d\d:\d\d ([A-Z]{3}?)')
                    .replace('EDT', 'EST5EDT'),
             str_date=(autos.createdOn.str.slice(4,19) + ' ' +
                       autos.createdOn.str.slice(-4)),
             createdOn=lambda df_: to_tz(df_, 'str_date',
                       'offset', 'America/New_York'),
             ffs=autos.eng_dscr.str.contains('FFS')
            )
     .pipe(err)
     .astype({'highway08': 'int8', 'city08': 'int16',
              'comb08': 'int16', 'fuelCost08': 'int16',
              'range': 'int16',  'year': 'int16',
              'make': 'category'})
     .drop(columns=['trany', 'eng_dscr'])
    )

In [ ]:
# replace err with this function
#from IPython.core.debugger import set_trace
#def err(*args):
#    set_trace()

In [ ]:
# running this will cause a ZeroDivisionError: division by zero
res = tweak_autos(autos)